### Stress Prediction all Models, Limited Features

This notebook uses sensor measurements from the
chest device: accelerometer data (ACC), temperature data 
(skin temperature), and electrodermal activity (EDA) aka galvanic skin response
(GSR).

The features of interest for the ACC data are as follows.

1. mean for each axis; summed over all axes 
2. STD for each axis; summed over all axes 
2. Peak frequency for x
3. Peak frequency for Y
4. Peak frequency for z

The features of interest for the temperature sensor are:

1. Min value
2. max value
2. Dynamic Range
4. Mean 
5. STD

The features of interest for the EDA data are:

1. Min value
2. max value
2. Dynamic Range
4. Mean 
5. STD


In [1]:
%pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importing all the necessary libraries

In [4]:
#from sklearn.datasets import fetch_mldata
import seaborn as sns
import sklearn as sklearn
from sklearn.neighbors import KNeighborsClassifier
import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
%pip install statsmodels 
import graphviz 
import statsmodels.api as sm
from sklearn import tree
import xgboost as xgb
from sklearn.decomposition import PCA

Note: you may need to restart the kernel to use updated packages.


In [5]:
#%matplotlib inline
%pip install pickle
%pip install numpy
%pip install pandas
%pip install matplotlib

import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
from sklearn.model_selection import train_test_split



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


Note: you may need to restart the kernel to use updated packages.




In [6]:
#data_set = r"C:\Users\dollj\Documents\Jupyter\WESAD\data\WESAD\S2"
#data_set = r"C:\Users\A\Desktop\neural networks\WESAD-data\S2"
s2_path = r"C:\Users\dollj\Documents\Jupyter\WESAD\data\WESAD\S2"
#s2_path = data_set + 'S2/S2.pkl'


with open(s2_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\dollj\\Documents\\Jupyter\\WESAD\\data\\WESAD\\S2'

In [ ]:
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Done")

In [ ]:
numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")



In [ ]:
display(df.shape)
display(df.info)
display(df.describe)

In [ ]:
# Calculating the interquartile range

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )

In [ ]:
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)


In [ ]:
#mean normalization 
norm_df_out=(df_out-df_out.mean())/df_out.std()
#min-max normalization:
#norm_df_out=(df_out-df_out.min())/(df_out.max()-df_out.min())

norm_y = df_out.w_label #keep original labels Dont normalize labels
norm_x = norm_df_out.drop('w_label',axis=1)


norm_x_train,norm_x_test,norm_y_train,norm_y_test=train_test_split(norm_x,norm_y,test_size=0.2)


In [ ]:
norm_df_out.head()

In [ ]:
norm_df_out.describe()

In [ ]:
norm_df_out.info()

In [ ]:
norm_df_out.shape

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,10))
cor = norm_x.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:

X_1 = sm.add_constant(norm_x)
#Fitting sm.OLS model
model = sm.OLS(norm_y,X_1).fit()
model.pvalues

In [ ]:
 pip install mlxtend

In [ ]:
#https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b
#https://towardsdatascience.com/feature-selection-using-wrapper-methods-in-python-f0d352b346f


from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# Sequential Forward Selection(sfs)
sfs = SFS(QuadraticDiscriminantAnalysis(),
           k_features=7,
           forward=True,
           floating=False,
           scoring = 'r2',
           cv = 0)
sfs.fit(norm_x, norm_y)
sfs.k_feature_names_     # to get the final set of features

In [ ]:
sfs = SFS(QuadraticDiscriminantAnalysis(),
           k_features=7,
           forward=False,
           floating=False,
           scoring = 'r2',
           cv = 0)
sfs.fit(norm_x, norm_y)
sfs.k_feature_names_     # to get the final set of features

In [ ]:
norm_x_train

In [ ]:
norm_y_train

In [ ]:
norm_x_test

In [ ]:
norm_y_test

In [ ]:

y=df_out.w_label
x=df_out.drop('w_label',axis=1)
#print(x)
#print("\n-----\n")
#print(y)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

y

In [ ]:
(x_train)


In [ ]:

(y_train)


In [ ]:
(x_test)


In [ ]:
(y_test)


# Logistic Regression Base Model

In [ ]:
clf = skl_lm.LogisticRegression(solver='newton-cg',penalty='none')
clf.fit(x_train,y_train)
y_out = clf.predict(x_test)
print(classification_report(y_test, y_out, digits=4))

'''
(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=3,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False) or use slover=saga
'''                 

# Logistic Regression + L2 regularization

In [ ]:
clf = skl_lm.LogisticRegression(solver='newton-cg',penalty='l2',multi_class='auto')
clf.fit(x_train,y_train)
y_out = clf.predict(x_test)
print(classification_report(y_test, y_out, digits=4))



In [ ]:
clf = skl_lm.LogisticRegression(solver='newton-cg',penalty='l2',multi_class='auto')
y_out = cross_val_predict(clf, norm_x, norm_y, cv=5)
print(classification_report(norm_y_test, y_out, digits=4))



# LDA Basic Implementation

In [ ]:
LDA= LinearDiscriminantAnalysis(solver = 'svd')
y_out = LDA.fit(x_train, y_train).predict(x_test)
#confusion_matrix(y_test, y_out)
print(classification_report(y_test, y_out, digits=4))#target_names=['Not Six', 'Six']

# LDA + Normalization + n_components &  5 Cross Validation

In [ ]:
LDA= LinearDiscriminantAnalysis(solver = 'svd')
#norm_y_out = LDA.fit(norm_x_train, norm_y_train).predict(norm_x_test)
norm_y_out = cross_val_predict(LDA, norm_x, norm_y, cv=5)
print(classification_report(norm_y_test, norm_y_out, digits=4))

# QDA Basic Implementation

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis()
qda_clf.fit(x_train,y_train)
y_out = qda_clf.fit(x_train, y_train).predict(x_test)
print(classification_report(y_test, y_out, digits=4))#target_names=['Not Six', 'Six']
#IF QDA is best, then data must be Gaussian

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis()
norm_y_out = cross_val_predict(qda_clf, norm_x, norm_y, cv=5)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# QDA + Normalization + Cross Validation = 10

# BEST MODEL *****++++****

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out = cross_val_predict(qda_clf, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)
'''
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html#sklearn.model_selection.cross_val_predict
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
'''    

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.5)
norm_y_out = cross_val_predict(qda_clf, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.9)
norm_y_out = cross_val_predict(qda_clf, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.7)
norm_y_out = cross_val_predict(qda_clf, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# QDA + Normalization + Cross Validation = 20

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis()
norm_y_out = cross_val_predict(qda_clf, norm_x, norm_y, cv=20)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

In [ ]:
'''
lm=(classification_report(y, y_out, digits=4))
print(lm)
scores = cross_val_score(qda_clf, x, y, cv=10, scoring='accuracy').mean()
print(scores)

#scoring='mean_squared_error'
'''

In [ ]:
'''
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

y_out = cross_val_predict(qda_clf, x, y, cv=loo)
lm=(classification_report(y, y_out, digits=4))
lm=(classification_report(y, y_out, digits=4))
print(lm)
'''

# KNN + Cross Validation + K=3 + Normalization [ CV = 10, K = 3 ]

In [ ]:
#Do base mdel with test and train data, show improvement in next section with cross validation
knn = KNeighborsClassifier(n_neighbors=3)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)
#https://www.ritchieng.com/machine-learning-cross-validation/

# KNN + Cross Validation + K=5 + Normalization [ CV = 20, K = 5 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=20)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# KNN + Cross Validation + K=10 + Normalization [ CV = 10, K = 10 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# KNN + Cross Validation [ CV = 10, K = 20 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=20)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# KNN + Cross Validation [ CV = 10, K = 30 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=30)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# KNN + Cross Validation [ CV = 10, K = 40 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=40)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# KNN + Cross Validation [ CV = 10, K = 60 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=60)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# KNN + Cross Validation [ CV = 10, K = 100 ]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=100)
norm_y_out = cross_val_predict(knn, norm_x, norm_y, cv=10)
lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# Simple Decision Tree + Depth = 5

In [ ]:

clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)

clf_i = clf.fit(x_train,y_train)

y_out_i = clf_i.predict(x_test)

lm_i=(classification_report(y_test, y_out_i, digits=4))
print(lm_i)

 

# Displaying and Drwaing the Tree

In [ ]:
tree.plot_tree(clf_i) 

In [ ]:
#from sklearn import tree
#https://scikit-learn.org/stable/modules/tree.html

features_ax=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp"]
classes_ay=['0', '1', '2', '3', '4']

dot_data = tree.export_graphviz(clf_i, out_file=None, 
                      feature_names=features_ax,  
                      class_names=classes_ay,  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render("Decision_tree")
graph 


# Simple Decision Tree + Depth = 5 + Cross Validation + Normalization

In [ ]:

clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)


norm_y_out = cross_val_predict(clf, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)


# Simple Decision Tree + Depth = 7 + Normalization + Cross Validation

In [ ]:

clf = DecisionTreeClassifier(criterion="entropy", max_depth=7)

norm_y_out = cross_val_predict(clf, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)


# Simple Decision Tree + Normalization + Cross Validation

## Criterion = Gini, Max_depth = 5, CV = 10

In [ ]:

clf = DecisionTreeClassifier(criterion="gini", max_depth=5)

norm_y_out = cross_val_predict(clf, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)


# XGB Implementation 

## Normalization + max_depth = 10, alpha = 10, n_estimators = 50, gamma=10

## colsample_bytree = 0.3, learning_rate = 0.1

In [ ]:
#https://towardsdatascience.com/https-medium-com-vishalmorde-xgboost-algorithm-long-she-may-rein-edd9f99be63d
#https://www.datacamp.com/community/tutorials/xgboost-in-python
#https://medium.com/@gabrielziegler3/multiclass-multilabel-classification-with-xgboost-66195e4d9f2d

xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)

xg_class.fit(x_train,y_train)

y_out = xg_class.predict(x_test)

lm_xgb=(classification_report(y_test, y_out, digits=4))

print(lm_xgb)


In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)


norm_y_out = cross_val_predict(xg_class, norm_x, norm_y, cv=5)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)


norm_y_out = cross_val_predict(xg_class, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# XGB Implementation 

## Normalization + max_depth = 5, alpha = 10, n_estimators = 20, gamma=20

## colsample_bytree = 0.3, learning_rate = 0.9

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.9,
                max_depth = 5, alpha = 10, n_estimators = 20, gamma=20)


norm_y_out = cross_val_predict(xg_class, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 70, gamma=10)


norm_y_out = cross_val_predict(xg_class, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# SVC + Normalization

In [ ]:
# Create a SVC classifier using a linear kernel
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1, random_state=0)
# Train the classifier
clf.fit(x_train, y_train)

#Predict the response for test dataset
y_out = clf.predict(x_test)
lm_svc=(classification_report(y_test, y_out, digits=4))
print(lm_svc)


# SVM + Normalization + Cross Validation

In [ ]:
##https://scikit-learn.org/stable/modules/svm.html
#https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python
#https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769
###https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
#https://chrisalbon.com/machine_learning/support_vector_machines/svc_parameters_using_rbf_kernel/

#svm = SVC(kernel='rbf', random_state=0, gamma=10, C=1)
#1000 iterations take 25 minutes
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='rbf', random_state=0, gamma=10, C=100,
              shrinking=True, tol=1.000, cache_size=10) ##(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(x_train, y_train)

#Predict the response for test dataset
y_out = clf.predict(x_test)
lm_svc=(classification_report(y_test, y_out, digits=4))
print(lm_svc)


# SVM + Normalization + Cross Validation + Gamma = 20

In [ ]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='rbf', random_state=0, gamma=20, C=100,
              shrinking=True, tol=1.000, cache_size=10) ##(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(norm_x_train, norm_y_train)

#Predict the response for test dataset
norm_y_out = clf.predict(norm_x_test)

In [ ]:
lm_svm=(classification_report(norm_y_test, norm_y_out, digits=4))
print(lm_svm)


In [ ]:
norm_y_out = cross_val_predict(clf, norm_x, norm_y, cv=10)

lm=(classification_report(norm_y, norm_y_out, digits=4))
print(lm)

# PCA Implementation with 4 features + KNN + XGBOOST

In [ ]:
#https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python


pca = PCA(n_components=4)

x_pca=pca.fit_transform(x)
#x_pca = pca.transform(x_pca)
print("original shape:   ", x.shape)
print("transformed shape:", x_pca.shape)

In [ ]:
y

In [ ]:
pca_df = pd.DataFrame(data=x_pca, columns=["PCA-1", "PCA-2", "PCA-3","PCA-4"]) 
pca_df['w_label'] = y.values
print("data frame combined",min(y),max(y))
print(pca_df.shape)

In [ ]:
(pca_df.head())
#print(pca_df["w_label"])

In [ ]:
pca_y = pca_df.w_label
pca_x = pca_df.drop('w_label',axis=1)

pca_x_train,pca_x_test, pca_y_train, pca_y_test = train_test_split(pca_x,pca_y,test_size=0.2)




In [ ]:
pca_x_train

In [ ]:
pca_y_train

In [ ]:
pca_x_test

In [ ]:
pca_y_test

# PCA with 4 features + Simple Decision Tree

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)

# Train Decision Tree Classifer
clf = clf.fit(pca_x_train, pca_y_train)

#Predict the response for test dataset
pca_y_out = clf.predict(pca_x_test)

# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_out))
lmIV=(classification_report(pca_y_test, pca_y_out, digits=4))
print(lmIV)


# PCA with 4 features + KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
pca_y_out = cross_val_predict(knn, pca_x, pca_y, cv=20)
lm=(classification_report(pca_y, pca_y_out, digits=4))
#https://www.ritchieng.com/machine-learning-cross-validation/
lm=(classification_report(pca_y, pca_y_out, digits=4))
print(lm)


# PCA with 4 features + XGBOOST

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)

xg_class.fit(pca_x_train,pca_y_train)

pca_y_out = xg_class.predict(pca_x_test)

lm_xgbIV=(classification_report(pca_y_test, pca_y_out, digits=4))

print(lm_xgbIV)

# PCA with 1 feature

In [ ]:
pca_df_1 = pca_df_2.drop('PCA-2',axis=1)
print(pca_df_1.head())

pca_y = pca_df_1.w_label
pca_x = pca_df_1.drop('w_label',axis=1)

pca_x_trainI,pca_x_testI, pca_y_trainI, pca_y_testI = train_test_split(pca_x,pca_y,test_size=0.2)
pca_x_trainI

# PCA with 1 feature + XGBOOST

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)

xg_class.fit(pca_x_trainI,pca_y_trainI)

pca_y_outI = xg_class.predict(pca_x_testI)

lm_xgbI=(classification_report(pca_y_testI, pca_y_outI, digits=4))

print(lm_xgbI)



# PCA with 2 features 

In [ ]:
pca_df_2 = pca_df_3.drop('PCA-3',axis=1)
print(pca_df_2.head())
pca_y = pca_df_2.w_label
pca_x = pca_df_2.drop('w_label',axis=1)

pca_x_trainII,pca_x_testII, pca_y_trainII, pca_y_testII = train_test_split(pca_x,pca_y,test_size=0.2)



In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)

# Train Decision Tree Classifer
clf = clf.fit(pca_x_trainII, pca_y_trainII)

#Predict the response for test dataset
pca_y_outII = clf.predict(pca_x_testII)

# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_out))
lm=(classification_report(pca_y_testII, pca_y_outII, digits=4))
print(lm)

# PCA with 2 features + XGBOOST

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)

xg_class.fit(pca_x_trainII,pca_y_trainII)

pca_y_outII = xg_class.predict(pca_x_testII)

lm_xgbII=(classification_report(pca_y_testII, pca_y_outII, digits=4))

print(lm_xgbII)


# PCA with 3 features

In [ ]:
pca_df_3 = pca_df.drop('PCA-4',axis=1)

print(pca_df_3.head())
pca_y = pca_df_3.w_label
pca_x = pca_df_3.drop('w_label',axis=1)

pca_x_trainIII,pca_x_testIII, pca_y_trainIII, pca_y_testIII = train_test_split(pca_x,pca_y,test_size=0.2)


# PCA with 3 features + XGBOOST

In [ ]:
xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50, gamma=10)

xg_class.fit(pca_x_trainIII,pca_y_trainIII)

pca_y_outIII = xg_class.predict(pca_x_testIII)

lm_xgbIII=(classification_report(pca_y_testIII, pca_y_outIII, digits=4))

print(lm_xgbIII)

In [ ]:
TESTING BEST MODEL ON 7 SUBJECTS 

In [ ]:


s3_path = data_set + 'S3/S3.pkl'

with open(s3_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')
    
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Equated_3")    

numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
norm_df_out=(df_out-df_out.mean())/df_out.std()

norm_y_3 = df_out.w_label #keep original labels Dont normalize labels
norm_x_3 = norm_df_out.drop('w_label',axis=1)


norm_x_train_3,norm_x_test_3,norm_y_train_3,norm_y_test_3=train_test_split(norm_x_3,norm_y_3,test_size=0.2)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out_3 = cross_val_predict(qda_clf, norm_x_3, norm_y_3, cv=10)
lm_3=(classification_report(norm_y_3, norm_y_out_3, digits=4))
print(lm_3)


In [ ]:


s4_path = data_set + 'S4/S4.pkl'

with open(s4_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')
    
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Equated")    

numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
norm_df_out=(df_out-df_out.mean())/df_out.std()

norm_y_4 = df_out.w_label #keep original labels Dont normalize labels
norm_x_4 = norm_df_out.drop('w_label',axis=1)


norm_x_train_4,norm_x_test_4,norm_y_train_4,norm_y_test_4=train_test_split(norm_x_4,norm_y_4,test_size=0.2)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out_4 = cross_val_predict(qda_clf, norm_x_4, norm_y_4, cv=10)
lm_4=(classification_report(norm_y_4, norm_y_out_4, digits=4))
print(lm_4)


In [ ]:


s10_path = data_set + 'S10/S10.pkl'

with open(s10_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')
    
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Equated")    

numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
norm_df_out=(df_out-df_out.mean())/df_out.std()

norm_y_10 = df_out.w_label #keep original labels Dont normalize labels
norm_x_10 = norm_df_out.drop('w_label',axis=1)


norm_x_train_10,norm_x_test_10,norm_y_train_10,norm_y_test_10=train_test_split(norm_x_10,norm_y_10,test_size=0.2)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out_10 = cross_val_predict(qda_clf, norm_x_10, norm_y_10, cv=10)
lm_10=(classification_report(norm_y_10, norm_y_out_10, digits=4))
print(lm_10)


In [ ]:


s11_path = data_set + 'S11/S11.pkl'

with open(s11_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')
    
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Equated")    

numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
norm_df_out=(df_out-df_out.mean())/df_out.std()

norm_y_11 = df_out.w_label #keep original labels Dont normalize labels
norm_x_11 = norm_df_out.drop('w_label',axis=1)


norm_x_train_11,norm_x_test_11,norm_y_train_11,norm_y_test_11=train_test_split(norm_x_11,norm_y_11,test_size=0.2)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out_11 = cross_val_predict(qda_clf, norm_x_11, norm_y_11, cv=10)
lm_11=(classification_report(norm_y_11, norm_y_out_11, digits=4))
print(lm_11)


In [ ]:


s16_path = data_set + 'S16/S16.pkl'

with open(s16_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')
    
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Equated")    

numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
norm_df_out=(df_out-df_out.mean())/df_out.std()

norm_y_16 = df_out.w_label #keep original labels Dont normalize labels
norm_x_16 = norm_df_out.drop('w_label',axis=1)


norm_x_train_16,norm_x_test_16,norm_y_train_16,norm_y_test_16=train_test_split(norm_x_16,norm_y_16,test_size=0.2)

In [ ]:
norm_x_16

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out_16 = cross_val_predict(qda_clf, norm_x_16, norm_y_16, cv=10)
lm_16=(classification_report(norm_y_16, norm_y_out_16, digits=4))
print(lm_16)


In [ ]:


s17_path = data_set + 'S17/S17.pkl'

with open(s17_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')
    
c_ax=s2_data['signal']['chest']['ACC'][0:,0]#[baseline_to_plot]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]#[baseline_to_plot]#[0:100] # hundred values
c_az=s2_data['signal']['chest']['ACC'][0:,2]#[baseline_to_plot]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
c_emg=s2_data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
c_eda=s2_data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
c_temp=s2_data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
c_resp=s2_data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
w_ax=s2_data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
w_ay=s2_data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
w_az=s2_data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
w_bvp=s2_data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
w_eda=s2_data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
w_temp=s2_data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
w_label=s2_data['label']

print("Equated")    

numpy_data1=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
numpy_data1=numpy_data1.T

#df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_ax","w_ay","w_az","w_bvp","w_eda","w_temp"],orient='index') 
df = pd.DataFrame(data=numpy_data1, columns=["c_ax", "c_ay", "c_az","c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 

print("data frame combined")


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print("IQR is\n", IQR)
#print((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)) )
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
norm_df_out=(df_out-df_out.mean())/df_out.std()

norm_y_17 = df_out.w_label #keep original labels Dont normalize labels
norm_x_17 = norm_df_out.drop('w_label',axis=1)


norm_x_train_17,norm_x_test_17,norm_y_train_17,norm_y_test_17=train_test_split(norm_x_17,norm_y_17,test_size=0.2)

In [ ]:
qda_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0)
norm_y_out_17 = cross_val_predict(qda_clf, norm_x_17, norm_y_17, cv=10)
lm_17=(classification_report(norm_y_17, norm_y_out_17, digits=4))
print(lm_17)
